In [ ]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from torch import nn
import numpy as np


In [ ]:
dt=pd.read_csv("train.csv")
dt.dropna(inplace=True)
dt.drop_duplicates(inplace=True)
dt

In [ ]:
dt=dt.iloc[:,1:]
dt_features=dt.values[:,:-1]
dt_features=torch.from_numpy(dt_features.astype('float32'))
dt_target=dt.values[:,-1]  ## convert pandas dataframe to numpy array along with slicing
dt_target=torch.from_numpy(LabelEncoder().fit_transform(dt_target))
dt



In [ ]:
train = torch.utils.data.TensorDataset(dt_features,dt_target)
trainloader=torch.utils.data.DataLoader(train,batch_size=80,shuffle=True)

In [ ]:
model = nn.Sequential( nn.Linear(75,64),
                       nn.ReLU(),
                      nn.Linear(64,32),
                      nn.ReLU(),
                      nn.Linear(32,9),
                      nn.LogSoftmax(dim=1)
                     )
criterion = nn.NLLLoss()
optimiser= torch.optim.SGD(model.parameters(),lr=0.005)

In [ ]:
model.load_state_dict(torch.load('check.pth'))

In [ ]:
data=iter(trainloader)
features , target = next(data)
model(features)
target.shape
loss=criterion(model(features),target)
loss

In [ ]:
for x in range(100):
  total_loss=0
  for features , target in trainloader:
    optimiser.zero_grad()
    loss=criterion(model(features),target)
    loss.backward()
    optimiser.step()
    total_loss = total_loss + loss.item()
  print(total_loss/len(trainloader))

In [ ]:
torch.save(model.state_dict(),'check.pth')

In [ ]:
test_data=pd.read_csv('test.csv')

In [ ]:
test_data=test_data.iloc[:,1:]
id=np.arange(200000,300000)
test_data=torch.from_numpy(test_data.to_numpy().astype('float32'))

In [ ]:
log_pr=model(test_data)
pr=torch.exp(log_pr)
pr

In [ ]:
data=pr.detach().numpy()
data=np.c_[id,data]
df=pd.DataFrame(data,columns=['id','Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])

In [ ]:
df.to_csv('result.csv')